In [1]:
import numpy as np
import json
import os
import copy
import pickle

import mesh_sampling
import trimesh
from shape_data import ShapeData
from SimplePointNet import SimplePointNet, PointNetAutoEncoder, Decoder, Encoder, SimpleTransformer

from autoencoder_dataset import autoencoder_dataset
from torch.utils.data import DataLoader

from spiral_utils import get_adj_trigs, generate_spirals
from models import SpiralAutoencoder
from train_funcs import train_autoencoder_dataloader
from test_funcs import test_autoencoder_dataloader


import torch
torch.cuda.empty_cache()
from tensorboardX import SummaryWriter

from sklearn.metrics.pairwise import euclidean_distances
meshpackage = 'trimesh' # 'mpi-mesh', 'trimesh'
root_dir = '../Data'

dataset = 'COMA'
name = ''

GPU = True
device_idx = 0
torch.cuda.get_device_name(device_idx)

'NVIDIA RTX A3000 Laptop GPU'

In [2]:
#T = np.load(os.path.join(root_dir, dataset, 'preprocessed_identity_pointnet_original', "train.npy"))

In [3]:
#T[0]

In [4]:
#a = np.dstack([T[0]]*19282).swapaxes(0,2).swapaxes(1,2)

In [5]:
#a = np.dstack([T[0]]*19282).swapaxes(0,2).swapaxes(1,2)

In [6]:
#np.save(os.path.join(root_dir, dataset, 'preprocessed_identity_pointnet_original', "train.npy"), a)

In [7]:
#B = np.load(os.path.join(root_dir, dataset, 'preprocessed_identity_pointnet_original', "points_train" ,"0.npy"))

In [8]:
#B

In [9]:
args = {}

generative_model = 'autoencoder'
downsample_method = 'COMA_downsample' # choose'COMA_downsample' or 'meshlab_downsample'

# below are the arguments for the DFAUST run
reference_mesh_file = os.path.join(root_dir, dataset, 'template', 'template.obj')
downsample_directory = os.path.join(root_dir, dataset,'template', downsample_method)
ds_factors = [4, 4, 4, 4]
step_sizes = [2, 2, 1, 1, 1]
filter_sizes_enc = [[64, 128], 64, [128, 64, 64]]
filter_sizes_dec = [128]
dilation_flag = True
if dilation_flag:
    dilation=[2, 2, 1, 1, 1] 
else:
    dilation = None
reference_points = [[3567,4051,4597]] #[[414]]  # [[3567,4051,4597]] used for COMA with 3 disconnected components

args = {'generative_model': generative_model,
        'name': name, 'data': os.path.join(root_dir, dataset, 'preprocessed_identity_pointnet_original',name),
        'results_folder':  os.path.join(root_dir, dataset,'results_identity/pointnet_original_'+ generative_model),
        'reference_mesh_file':reference_mesh_file, 'downsample_directory': downsample_directory,
        'checkpoint_file': 'checkpoint',
        'seed':2, 'loss':'varifold',
        'batch_size':8, 'num_epochs':120, 'eval_frequency':200, 'num_workers': 4,
        'filter_sizes_enc': filter_sizes_enc, 'filter_sizes_dec': filter_sizes_dec,
        'nz':128, 
        'ds_factors': ds_factors, 'step_sizes' : step_sizes, 'dilation': dilation,
        'lr':1e-3, 
        'regularization': 5e-5,
        'scheduler': True, 'decay_rate': 0.99,'decay_steps':1,  
        'resume': False,
        
        'mode':'train', 'shuffle': True, 'nVal': 100, 'normalization': True}

args['results_folder'] = os.path.join(args['results_folder'],'latent_'+str(args['nz']))
    
if not os.path.exists(os.path.join(args['results_folder'])):
    os.makedirs(os.path.join(args['results_folder']))

summary_path = os.path.join(args['results_folder'],'summaries',args['name'])
if not os.path.exists(summary_path):
    os.makedirs(summary_path)  
    
checkpoint_path = os.path.join(args['results_folder'],'checkpoints', args['name'])
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
samples_path = os.path.join(args['results_folder'],'samples', args['name'])
if not os.path.exists(samples_path):
    os.makedirs(samples_path)
    
prediction_path = os.path.join(args['results_folder'],'predictions', args['name'])
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)

if not os.path.exists(downsample_directory):
    os.makedirs(downsample_directory)

### Normalize data

In [10]:
np.random.seed(args['seed'])
print("Loading data .. ")
if not os.path.exists(args['data']+'/mean.npy') or not os.path.exists(args['data']+'/std.npy'):
    shapedata =  ShapeData(nVal=args['nVal'], 
                          train_file=args['data']+'/train.npy', 
                          test_file=args['data']+'/test.npy', 
                          reference_mesh_file=args['reference_mesh_file'],
                          normalization = args['normalization'],
                          meshpackage = meshpackage, load_flag = True, mean_subtraction_only = False)
    np.save(args['data']+'/mean.npy', shapedata.mean)
    np.save(args['data']+'/std.npy', shapedata.std)
else:
    shapedata = ShapeData(nVal=args['nVal'], 
                         train_file=args['data']+'/train.npy',
                         test_file=args['data']+'/test.npy', 
                         reference_mesh_file=args['reference_mesh_file'],
                         normalization = args['normalization'],
                         meshpackage = meshpackage, load_flag = False, mean_subtraction_only = False)
    shapedata.mean = np.load(args['data']+'/mean.npy')
    shapedata.std = np.load(args['data']+'/std.npy')
    shapedata.n_vertex = shapedata.mean.shape[0]
    shapedata.n_features = shapedata.mean.shape[1]
    
print("... Done")    

Loading data .. 
... Done


In [11]:
torch.manual_seed(args['seed'])

if GPU:
    device = torch.device("cuda:"+str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cuda:0


In [12]:
# Building model, optimizer, and loss function

dataset_train = autoencoder_dataset(root_dir = args['data'], points_dataset = 'train',
                                           shapedata = shapedata,
                                           normalization = args['normalization'])

dataloader_train = DataLoader(dataset_train, batch_size=args['batch_size'],\
                                     shuffle = args['shuffle'], num_workers = args['num_workers'])

dataset_val = autoencoder_dataset(root_dir = args['data'], points_dataset = 'val', 
                                         shapedata = shapedata,
                                         normalization = args['normalization'])

dataloader_val = DataLoader(dataset_val, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'])

dataset_test = autoencoder_dataset(root_dir = args['data'], points_dataset = 'test',
                                          shapedata = shapedata,
                                          normalization = args['normalization'])

dataloader_test = DataLoader(dataset_test, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'])


if 'autoencoder' in args['generative_model']:
        model = PointNetAutoEncoder(latent_size=args['nz'],
                                    filter_enc = args['filter_sizes_enc'], 
                                    filter_dec = args['filter_sizes_dec'],  
                                    num_points=shapedata.n_vertex+1, 
                                    device=device).to(device)                  


optim = torch.optim.Adam(model.parameters(),lr=args['lr'],weight_decay=args['regularization'])
if args['scheduler']:
    scheduler=torch.optim.lr_scheduler.StepLR(optim, args['decay_steps'],gamma=args['decay_rate'])
else:
    scheduler = None

64


### Loss functions

In [13]:
template_mesh = trimesh.load("../Data/COMA/template/template.obj")
faces = torch.Tensor(template_mesh.faces)
mesh1 = trimesh.load_mesh("../datasets/COMA/FaceTalk_170725_00137_TA/bareteeth/bareteeth.000001.ply")
mesh2 = trimesh.load_mesh("../datasets/COMA/FaceTalk_170728_03272_TA/bareteeth/bareteeth.000001.ply")

In [14]:
torchdtype = torch.float

from pykeops.torch import LazyTensor

def computeNormalsCenters(vertices, faces):
    """
    Function that computes centers and surface elements (area * normal) of triangles of a mesh
    :np.array, Nvx3 vertices: vertices of the mesh
    :np.array, NFx3 faces: faces of the mesh
    :return np.array, NFx3 centers np.array , NFx3 surfel: centers and surface elements of the mesh
    """
    xDef1 = torch.Tensor(vertices[faces[:, 0], :])
    xDef2 = torch.Tensor(vertices[faces[:, 1], :])
    xDef3 = torch.Tensor(vertices[faces[:, 2], :])
    
    centers, normals = (xDef1 + xDef2 + xDef3) / 3, 0.5 * torch.cross(xDef2 - xDef1, xDef3 - xDef1)
    areas = (normals ** 2).sum(dim=1)[:, None].sqrt()
    
    return centers, normals, np.squeeze(areas)

def lazy_varifold_dist_batch(areas1, normals1, centers1, areas2, normals2, centers2, sigma):
    """
    Pykeops function to compute absolute varifolds product over 2 batches of meshes (center + surface elements)
    :np.array, N_1x3 surfels1: surfels of batch number 1
    :np.array, N_1x3 centers1: centers of batch number 1
    :np.array, N_2x3 surfels2: surfels of batch number 2
    :np.array, N_2x3 centers2: centers of batch number 2
    :float sigma: sigma parameter of gaussian kernel
    :return np.array N_1xN_2 batch of absolute products:
    """
    # Current, and absolute varifolds can be defined directly from surface elements (normals * areas)
    surfels1 = areas1[:, :, None]*normals1
    surfels2 = areas2[:, :, None]*normals2
    # Transforming into Pykeops lazytensors (no direct evaluation, compute during compilation)
    surfe_laz_1 = LazyTensor(surfels1[:, None, :, None, :].contiguous().to(device))
    center_laz_1 = LazyTensor(centers1[:, None, :, None, :].contiguous().to(device))
    surfe_laz_2 = LazyTensor(surfels2[None, :, None, :, :].contiguous().to(device))
    center_laz_2 = LazyTensor(centers2[None, :, None, :, :].contiguous().to(device))
    # Computing absolute varifolds product
    gamma = 1 / (sigma * sigma)
    D2 = ((center_laz_1 - center_laz_2) ** 2).sum()
    K = (-D2 * gamma).exp() * (surfe_laz_1 * surfe_laz_2).sum().abs()
    final_K = K.sum_reduction(dim=3)
    return final_K.sum(dim=[2,3])

def lazy_varifold_or_dist_batch(areas1, normals1, centers1, areas2, normals2, centers2, sigma):
    """
    Pykeops function to compute oriented varifolds product over 2 batches of meshes (center + surface elements)
    :np.array, N_1x3 surfels1: surfels of batch number 1
    :np.array, N_1x3 centers1: centers of batch number 1
    :np.array, N_2x3 surfels2: surfels of batch number 2
    :np.array, N_2x3 centers2: centers of batch number 2
    :float sigma: sigma parameter of gaussian kernel
    :return np.array N_1xN_2 batch of oriented varifolds products:
    """
    # Transforming into Pykeops lazytensors (no direct evaluation, compute during compilation)
    area_laz_1 = LazyTensor(areas1[:, None, :, None, None].contiguous().to(device))
    norm_laz_1 = LazyTensor(normals1[:, None, :, None, :].contiguous().to(device))
    center_laz_1 = LazyTensor(centers1[:, None, :, None, :].contiguous().to(device))
    area_laz_2 = LazyTensor(areas2[None, :, None, :, None].contiguous().to(device))
    norm_laz_2 = LazyTensor(normals2[None, :, None, :, :].contiguous().to(device))
    center_laz_2 = LazyTensor(centers2[None, :, None, :, :].contiguous().to(device))
    # Computing oriented varifolds product
    gamma = 1 / (sigma * sigma)
    D2 = ((center_laz_1 - center_laz_2) ** 2).sum()
    K = (-D2 * gamma).exp() * ((norm_laz_1 * norm_laz_2).sum()/0.5).exp() * (area_laz_1* area_laz_2).sum()
    final_K = K.sum_reduction(dim=3)
    return final_K.sum(dim=[2,3])

def compute_feats(vertices, faces, do_cm=False):
    """
    Computing centers and surface element of a mesh, converting them to Pytorch tensors. Centroid normalization if needed
    :np.array Nvx3 vertices: vertices of the mesh
    :np.array Nfx3 faces: faces of the mesh
    :bool do_cm: centroid normalization or not
    :return (torch.Tensor, Nfx3), (torch.Tensor, Nfx3), (np.array, 3): centers tensor, surfel tensor, centroid (None if not needed)
    """
    centers, normals, areas = computeNormalsCenters(vertices, faces)
    cm = None
    if do_cm:
        cm = np.average(centers, weights=areas, axis=0)
        centers = centers-cm
    return centers.float(), normals.float(), areas.float(), cm


def prepare_batch(vertices_list, faces_list, centroid=False):
    """
    Computing batched centers and surface element of a list of meshes, converting them to Pytorch tensors. Centroid normalization if needed
    :np.array NbxNvx3 vertices: Batched vertices of the meshes
    :np.array NbxNfx3 faces: Batched faces of the meshes
    :bool do_cm: centroid normalization or not
    :return (torch.Tensor, NbxNfx3), (torch.Tensor, NbxNfx3), (np.array, Nbx3): batched centers tensor, surfel tensor, centroid (None if not needed)
    """
    Ns = [f.shape[0] for f in faces_list]
    N = max(Ns)
    batch_areas = torch.zeros((len(faces_list), N))
    batch_normals = torch.zeros((len(faces_list), N, 3))
    batch_centers = torch.zeros((len(faces_list), N, 3))
    centroids = []
    for i in range(len(faces_list)):
        N_i = faces_list[i].shape[0]
        batch_centers[i, :N_i, :], batch_normals[i, :N_i, :], batch_areas[i, :N_i], cm = compute_feats(vertices_list[i], faces_list[i], centroid)
        centroids.append(cm)
    if not centroid:
        centroids = None
    return batch_areas, batch_normals, batch_centers, centroids

#F1, V1 = torch.Tensor(template_mesh.faces).to(dtype=torch.int32, device='cpu'), torch.Tensor(template_mesh.vertices)

c1, n1, a1, _ = compute_feats(template_mesh.vertices, template_mesh.faces)

faces_list1 = torch.Tensor(mesh1.faces).to(dtype=torch.int32, device='cpu').repeat(args['batch_size'],1,1).numpy()
vertices_list1 = torch.Tensor(mesh1.vertices).repeat(args['batch_size'],1,1).numpy()

faces_list2 = torch.Tensor(mesh2.faces).to(dtype=torch.int32, device='cpu').repeat(args['batch_size'],1,1).numpy()
vertices_list2 = torch.Tensor(mesh2.vertices).repeat(args['batch_size'],1,1).numpy()

a1, n1, c1, _  = prepare_batch(vertices_list1, faces_list1)
a2, n2, c2, _  = prepare_batch(vertices_list2, faces_list2)

In [15]:
#batch_areas, batch_normals, batch_centers, centroids = prepare_batch(vertices_list, faces_list)

mu1 = lazy_varifold_or_dist_batch(a1, n1, c1, a1, n1, c1, sigma=1)
mu2 = lazy_varifold_or_dist_batch(a2, n2, c2, a2, n2, c2, sigma=1)
mu1_2 = lazy_varifold_or_dist_batch(a2, n2, c2, a1, n1, c1, sigma=1)

(mu1 + mu2 - 2*mu1_2).sqrt()

if args['loss']=='varifold2':
    
    from pykeops.torch import LazyTensor
    # PyKeOps counterpart
    KeOpsdeviceId = device.index  # id of Gpu device (in case Gpu is  used)
    KeOpsdtype = torchdtype.__str__().split(".")[1]  # 'float32'
    
    #new_faces = torch.Tensor(faces).to(dtype=torch.int32, device=device).repeat(args['batch_size'],1,1)
    def loss_varifold(outputs, targets):
        
        new_faces = torch.Tensor(faces).to(dtype=torch.long, device=device).repeat(outputs.shape[0],1,1)
        a1, n1, c1, _ = prepare_batch(outputs, new_faces, centroid=False)
        a2, n2, c2, _ = prepare_batch(targets, new_faces, centroid=False)
        L = lazy_varifold_or_dist_batch(a1, n1, c1, a1, n1, c1, sigma=1) + lazy_varifold_or_dist_batch(a2, n2, c2, a2, n2, c2, sigma=1) - 2*(lazy_varifold_or_dist_batch(a2, n2, c2, a1, n1, c1, sigma=1))
        
        return L.sqrt().mean()
    loss_fn = loss_varifold

In [16]:
sigma = 0.1
sigma = torch.tensor([sigma], dtype=torchdtype, device=device)
if args['loss']=='varifold':
    
    import lddmm_utils
    # PyKeOps counterpart
    KeOpsdeviceId = device.index  # id of Gpu device (in case Gpu is  used)
    KeOpsdtype = torchdtype.__str__().split(".")[1]  # 'float32'
    
    new_faces = faces.repeat(args['batch_size'],1,1)
    def loss_varifold(outputs, targets):
        new_faces = faces.repeat(outputs.shape[0],1,1)
        V1, F1 = outputs.to(dtype=torchdtype, device=device).requires_grad_(True), new_faces.to(dtype=torch.int32, device=device)
        V2, F2 = targets.to(dtype=torchdtype, device=device).requires_grad_(True), new_faces.to(dtype=torch.int32, device=device)
        
        L = torch.Tensor([lddmm_utils.lossVarifoldSurf(F1[i], V2[i], F2[i],
                          lddmm_utils.GaussLinKernel(sigma=sigma))(V1[i]) for i in range(new_faces.shape[0])]).requires_grad_(True).mean()
        
        return L.sqrt()
    loss_fn = loss_varifold

In [17]:
if args['loss']=='chamfer':
    from chamfer_distance import ChamferDistance as chamfer_dist
    def loss_chamfer(outputs, targets):
        dist1, dist2, idx1, idx2 = chamfer_dist(outputs,targets)
        loss = (torch.mean(dist1)) + (torch.mean(dist2))
        return loss
    loss_fn = loss_chamfer

In [18]:
if args['loss']=='l1':
    def loss_l1(outputs, targets):
        #print(outputs)
        L = torch.abs(outputs - targets).mean()
        return L 
    loss_fn = loss_l1

In [19]:
if args['loss']=='chamfer_varifold':
    
    import lddmm_utils
    from chamfer_distance import ChamferDistance as chamfer_dist
    
    # PyKeOps counterpart
    KeOpsdeviceId = device.index  # id of Gpu device (in case Gpu is  used)
    KeOpsdtype = torchdtype.__str__().split(".")[1]  # 'float32'
    
    new_faces = faces.repeat(args['batch_size'],1,1)
    def loss_varifold(outputs, targets):
        new_faces = faces.repeat(outputs.shape[0],1,1)
        V1, F1 = outputs.to(dtype=torchdtype, device=device).requires_grad_(True), new_faces.to(dtype=torch.int32, device=device)
        V2, F2 = targets.to(dtype=torchdtype, device=device).requires_grad_(True), new_faces.to(dtype=torch.int32, device=device)
        
        L = torch.Tensor([lddmm_utils.lossVarifoldSurf(F1[i], V2[i], F2[i],
                          lddmm_utils.GaussLinKernel(sigma=sigma))(V1[i]) for i in range(new_faces.shape[0])]).sqrt().mean().requires_grad_(True)
        
        return L
    
    def loss_chamfer(outputs, targets):
        dist1, dist2, idx1, idx2 = chamfer_dist(outputs,targets)
        loss = (torch.mean(dist1)) + (torch.mean(dist2))
        return loss
    
    def loss_chamfer_varifold(outputs, targets):
        return(loss_chamfer(outputs, targets)*2 + loss_varifold(outputs,targets)/200)
    
    loss_fn = loss_chamfer_varifold

In [20]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters is: {}".format(params)) 
print(model)
# print(M[4].v.shape)

Total number of parameters is: 2217385
PointNetAutoEncoder(
  (encoder): Encoder(
    (ptnet): SimplePointNet(
      (conv_layers): ModuleList(
        (0): Sequential(
          (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (1): Sequential(
          (0): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (2): Sequential(
          (0): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (transformers): ModuleList(
        (0): SimpleTransformer(
          (conv_layers): ModuleList(
            (0): Sequential(
              (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
           

In [21]:
if args['mode'] == 'train':
    writer = SummaryWriter(summary_path)
    with open(os.path.join(args['results_folder'],'checkpoints', args['name'] +'_params.json'),'w') as fp:
        saveparams = copy.deepcopy(args)
        json.dump(saveparams, fp)
        
    if args['resume']:
            print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file'])))
            checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
            start_epoch = checkpoint_dict['epoch'] + 1
            model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
            optim.load_state_dict(checkpoint_dict['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint_dict['scheduler_state_dict'])
            print('Resuming from epoch %s'%(str(start_epoch)))     
    else:
        start_epoch = 0
        
    if args['generative_model'] == 'autoencoder':
        train_autoencoder_dataloader(dataloader_train, dataloader_val,
                          device, model, optim, loss_fn,
                          bsize = args['batch_size'],
                          start_epoch = start_epoch,
                          n_epochs = args['num_epochs'],
                          eval_freq = args['eval_frequency'],
                          scheduler = scheduler,
                          writer = writer,
                          save_recons=True,
                          shapedata=shapedata,
                          metadata_dir=checkpoint_path, samples_dir=samples_path,
                          checkpoint_path = args['checkpoint_file'])

  0%|                                                                                                                                                                                                  | 0/187 [00:00<?, ?it/s]

[KeOps] Generating code for formula Sum_Reduction((Exp(-(Var(1,3,1)-Var(0,3,0))**2*Var(5,1,2))*Sum(Var(2,3,0)*Var(3,3,1)))*Var(4,1,1),0) ... OK


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 25.55it/s]


epoch 0 | tr 17.231910968737083 | val 24.45854820251465


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 25.03it/s]


...

epoch 117 | tr 17.182577142121403 | val 25.987719497680665


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 19.08it/s]


epoch 118 | tr 17.171284713285207 | val 30.572278060913085


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 20.06it/s]

epoch 119 | tr 17.210068260034458 | val 27.54523582458496
~FIN~


In [22]:
args['mode']='test'
if args['mode'] == 'test':
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    targets, predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_test, 
                                                                     shapedata, mm_constant = 1000)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)
    np.save(os.path.join(prediction_path,'targets'), targets)
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file ../Data/COMA/results_identity/pointnet_original_autoencoder/latent_128/checkpoints/checkpoint.pth.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 148/148 [00:01<00:00, 107.65it/s]


autoencoder: normalized loss 9.693267822265625
autoencoder: euclidean distance in mm= 28.097780227661133


In [1]:
from geomloss import SamplesLoss

In [2]:
Loss = SamplesLoss("sinkhorn", p=1, blur=0.01, scaling=0.9, truncate=1) #p=1 ou p=2

In [3]:
from plyfile import PlyData, PlyElement


def load_ply_file(fname):
    """Loads a .ply mesh to return a collection of weighted Dirac atoms: one per triangle face."""

    # Load the data, and read the connectivity information:
    plydata = PlyData.read(fname)
    triangles = np.vstack(plydata["face"].data["vertex_indices"])

    # Normalize the point cloud, as specified by the user:
    points = np.vstack([[v[0], v[1], v[2]] for v in plydata["vertex"]])

    return to_measure(points, triangles)

In [4]:
def to_measure(points, triangles):
    """Turns a triangle into a weighted point cloud."""

    # Our mesh is given as a collection of ABC triangles:
    A, B, C = points[triangles[:, 0]], points[triangles[:, 1]], points[triangles[:, 2]]

    # Locations and weights of our Dirac atoms:
    X = (A + B + C) / 3  # centers of the faces
    S = np.sqrt(np.sum(np.cross(B - A, C - A) ** 2, 1)) / 2  # areas of the faces

    print(
        "File loaded, and encoded as the weighted sum of {:,} atoms in 3D.".format(
            len(X)
        )
    )

    # We return a (normalized) vector of weights + a "list" of points
    return torch.Tensor(S / np.sum(S)), torch.Tensor(X)

In [13]:
pc1 = load_ply_file("../datasets/COMA/FaceTalk_170725_00137_TA/bareteeth/bareteeth.000001.ply")
pc2 = load_ply_file("../datasets/COMA/FaceTalk_170728_03272_TA/bareteeth/bareteeth.000001.ply")

File loaded, and encoded as the weighted sum of 9,976 atoms in 3D.
File loaded, and encoded as the weighted sum of 9,976 atoms in 3D.


In [14]:
a, x = pc1  # weights, locations
b, y = pc2 

x.requires_grad = True
z = x.clone()  # Moving point cloud
Loss(a,z,b,y)

tensor(0.0021, grad_fn=<SelectBackward0>)